In [1]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

In [2]:
from konlpy.tag import Mecab

In [3]:
mecab = Mecab()
print(mecab.morphs("강남역에 발롱펌 잘하는 곳 알려줘"))
print(mecab.morphs("이태원에 미용실 싼 곳 알아?"))

['강남역', '에', '발', '롱', '펌', '잘', '하', '는', '곳', '알려줘']
['이태원', '에', '미용실', '싼', '곳', '알', '아', '?']


### 아래와 같이 태그를 인식해야 한다.
- 장소 태그 : LOC (지역, 지하철역)
- 스타일 태그 : STY (헤어스타일)
- 형용사 태그 : ADJ (스타일이나 미용실을 표현하는 형용사)
- 업소명 태그 : ORG (미용실 이름)
- 기타 태그 : O (Out of tag)

['강남역', '에', '발', '롱', '펌', '잘', '하', '는', '곳', '알려줘']<br/>
['B-LOC', 'O', 'B-STY', 'I-STY', 'I-STY', 'B-ADJ', 'I-AJD', 'I-AJD', 'O', 'O']<br/><br/>
['이태원', '에', '미용실', '싼', '곳', '알', '아', '?']<br/>
['B-LOC', 'O', 'O', 'B-ADJ', 'O', 'O', 'O', 'O']

---
# 1. 질문 조각 전처리
    - 1) 형용사
        - 1-1) 전체 형용사
        - 1-2) 위치 형용사
    - 2) 스타일
    - 3) 장소
        - 3-1) 구, 동 
        - 3-2) 지하철역
    - 4) 업소명
    - 5) 어미

## 1) 형용사

In [4]:
adj_list = pd.read_excel('./input_data/adjectives_for_questions.xlsx')
adj_list

,san_pos_keywords,cs_pos_keywords,mood_pos_keywords,price_pos_keywords,loc_pos_keywords,sat_pos_keywords,etc_keywords
0,깔끔한,친절한,분위기가 좋은,싼,가까운,인스타각의,결혼식용으로 할
1,깨끗한,불친절하지 않은,화사한,가격이 저렴한,가까운 곳인,그뤠잇한,상견례용으로 할
2,청결한,친절한,넓은,혜자스러운,위치가 좋은,개쩌는,입학식용으로 할
3,위생적인,서비스가 좋은,넓직한,가격이 착한,접근성이 좋은,대만족의,졸업식때 할
4,빛나는,서비스 잘해주는,모던한,착한 가격의,금방 걸어갈수 있는,만족할만한,입학식때 할
...,...,...,...,...,...,...,...
90,NaN,NaN,NaN,NaN,NaN,이쁜,NaN
91,NaN,NaN,NaN,NaN,NaN,두상에 맞는,NaN
92,NaN,NaN,NaN,NaN,NaN,머리 빨리 해주는,NaN
93,NaN,NaN,NaN,NaN,NaN,핫한,NaN


In [5]:
adjs_sanitation = adj_list.san_pos_keywords.dropna().tolist()
adjs_customer_service = adj_list.cs_pos_keywords.dropna().tolist()
adjs_mood = adj_list.mood_pos_keywords.dropna().tolist()
adjs_price = adj_list.price_pos_keywords.dropna().tolist()
adjs_loc = adj_list.loc_pos_keywords.dropna().tolist()
adjs_satisfaction = adj_list.sat_pos_keywords.dropna().tolist()

len(adjs_sanitation), len(adjs_customer_service), len(adjs_mood), len(adjs_price), len(adjs_loc), len(adjs_satisfaction)

(22, 20, 32, 26, 22, 95)

In [6]:
adjs_sanitation = list(set(adjs_sanitation))
adjs_customer_service = list(set(adjs_customer_service))
adjs_mood = list(set(adjs_mood))
adjs_price = list(set(adjs_price))
adjs_loc = list(set(adjs_loc))
adjs_satisfaction = list(set(adjs_satisfaction))

In [7]:
adjs_all = adjs_sanitation + adjs_customer_service + adjs_mood  + adjs_price + adjs_loc + adjs_satisfaction
print('형용사 총 개수 :', len(adjs_all))

형용사 총 개수 : 208


### 1-2) 전체 형용사

In [8]:
df_adjs_all = DataFrame({'adj_all' : adjs_all})
df_adjs_all

,adj_all
0,비위생적이지 않은
1,반짝이는
2,쾌적한
3,빛나는
4,상쾌한
...,...
203,실력있는
204,전문 디자이너가 많은
205,단정해보이는
206,칭찬할만한


In [9]:
df_adjs_all['mecab'] = df_adjs_all.adj_all.apply(mecab.morphs)
df_adjs_all

,adj_all,mecab
0,비위생적이지 않은,"[비, 위생, 적, 이, 지, 않, 은]"
1,반짝이는,"[반짝이, 는]"
2,쾌적한,"[쾌적, 한]"
3,빛나는,"[빛나, 는]"
4,상쾌한,"[상쾌, 한]"
...,...,...
203,실력있는,"[실력, 있, 는]"
204,전문 디자이너가 많은,"[전문, 디자이너, 가, 많, 은]"
205,단정해보이는,"[단정, 해, 보이, 는]"
206,칭찬할만한,"[칭찬, 할, 만, 한]"


In [10]:
from collections import defaultdict
adj_tag_dict = defaultdict(lambda: 'I-ADJ')
adj_tag_dict[0] = 'B-ADJ'
adj_tag_dict

defaultdict(<function __main__.<lambda>()>, {0: 'B-ADJ'})

In [11]:
def get_adj_BIO_tag(morphs: list):
    return [adj_tag_dict[i] for i in range(len(morphs))]

In [12]:
df_adjs_all['BIO_tag'] = df_adjs_all.mecab.apply(get_adj_BIO_tag)
df_adjs_all.head()

,adj_all,mecab,BIO_tag
0,비위생적이지 않은,"[비, 위생, 적, 이, 지, 않, 은]","[B-ADJ, I-ADJ, I-ADJ, I-ADJ, I-ADJ, I-ADJ, I-ADJ]"
1,반짝이는,"[반짝이, 는]","[B-ADJ, I-ADJ]"
2,쾌적한,"[쾌적, 한]","[B-ADJ, I-ADJ]"
3,빛나는,"[빛나, 는]","[B-ADJ, I-ADJ]"
4,상쾌한,"[상쾌, 한]","[B-ADJ, I-ADJ]"


### 1-2) 위치 형용사

In [13]:
df_adjs_loc = DataFrame({'adjs_loc' : adjs_loc})
df_adjs_loc

,adjs_loc
0,가까운
1,걸어서 10분 거리에 있는
2,금방 걸어가는
3,근처
4,근처에
5,가까운 곳인
6,접근성이 좋은
7,근처의
8,멀지 않은
9,위치가 좋은


In [14]:
df_adjs_loc['mecab'] = df_adjs_loc.adjs_loc.apply(mecab.morphs)
df_adjs_loc

,adjs_loc,mecab
0,가까운,[가까운]
1,걸어서 10분 거리에 있는,"[걸어서, 10, 분, 거리, 에, 있, 는]"
2,금방 걸어가는,"[금방, 걸어가, 는]"
3,근처,[근처]
4,근처에,"[근처, 에]"
5,가까운 곳인,"[가까운, 곳, 인]"
6,접근성이 좋은,"[접근성, 이, 좋, 은]"
7,근처의,"[근, 처의]"
8,멀지 않은,"[멀, 지, 않, 은]"
9,위치가 좋은,"[위치, 가, 좋, 은]"


In [15]:
df_adjs_loc['BIO_tag'] = df_adjs_loc.mecab.apply(get_adj_BIO_tag)
df_adjs_loc.head()

,adjs_loc,mecab,BIO_tag
0,가까운,[가까운],[B-ADJ]
1,걸어서 10분 거리에 있는,"[걸어서, 10, 분, 거리, 에, 있, 는]","[B-ADJ, I-ADJ, I-ADJ, I-ADJ, I-ADJ, I-ADJ, I-ADJ]"
2,금방 걸어가는,"[금방, 걸어가, 는]","[B-ADJ, I-ADJ, I-ADJ]"
3,근처,[근처],[B-ADJ]
4,근처에,"[근처, 에]","[B-ADJ, I-ADJ]"


## 2) 스타일

In [16]:
styles = ['댄디컷','리젠트컷','스왓컷','크롭컷','포마드컷','쉼표머리'
        ,'투블럭','모히칸','머쉬룸컷','레이어드컷','울프컷','언밸런스컷'
        ,'히메컷','보브컷','샤기컷','허쉬컷','처피뱅','러블리펌','C컬펌'
        ,'S컬펌','물결펌','히피펌','글램펌','트위스트펌','발롱펌'
        ,'베이비펌','바디펌','루즈펌','가르마펌','쉐도우펌','포마드펌'
        ,'스핀스왈로우펌','애즈펌','리젠트펌','다운펌','드레드락','호일펌'
        ,'볼륨매직펌','매직펌','에어펌','내츄럴펌','바그펌','보니펌','레인펌'
        ,'프릴펌','페미닌펌','이지펌','아이롱펌','보울컷','리프컷','고데기펌'
        ,'웨이브펌','복구펌','젤리펌','볼륨매직','픽시컷','보브컷','언발란스컷'
        ,'가일컷','가일펌','그레이스펌','태슬컷','디자인펌','붙임머리','셋팅펌'
        ,'엘리자벳펌','리프펌','스핀스왈로펌','시스루뱅','투블럭펌', '버즈컷', '레게머리', '귀두컷', '해병대머리', '반삭', '박새로이컷']

In [17]:
styles.sort()
styles

['C컬펌',
 'S컬펌',
 '가르마펌',
 '가일컷',
 '가일펌',
 '고데기펌',
 '귀두컷',
 '그레이스펌',
 '글램펌',
 '내츄럴펌',
 '다운펌',
 '댄디컷',
 '드레드락',
 '디자인펌',
 '러블리펌',
 '레게머리',
 '레이어드컷',
 '레인펌',
 '루즈펌',
 '리젠트컷',
 '리젠트펌',
 '리프컷',
 '리프펌',
 '매직펌',
 '머쉬룸컷',
 '모히칸',
 '물결펌',
 '바그펌',
 '바디펌',
 '박새로이컷',
 '반삭',
 '발롱펌',
 '버즈컷',
 '베이비펌',
 '보니펌',
 '보브컷',
 '보브컷',
 '보울컷',
 '복구펌',
 '볼륨매직',
 '볼륨매직펌',
 '붙임머리',
 '샤기컷',
 '셋팅펌',
 '쉐도우펌',
 '쉼표머리',
 '스왓컷',
 '스핀스왈로우펌',
 '스핀스왈로펌',
 '시스루뱅',
 '아이롱펌',
 '애즈펌',
 '언발란스컷',
 '언밸런스컷',
 '에어펌',
 '엘리자벳펌',
 '울프컷',
 '웨이브펌',
 '이지펌',
 '젤리펌',
 '처피뱅',
 '크롭컷',
 '태슬컷',
 '투블럭',
 '투블럭펌',
 '트위스트펌',
 '페미닌펌',
 '포마드컷',
 '포마드펌',
 '프릴펌',
 '픽시컷',
 '해병대머리',
 '허쉬컷',
 '호일펌',
 '히메컷',
 '히피펌']

In [18]:
df_styles = DataFrame({'style' : styles})
df_styles

,style
0,C컬펌
1,S컬펌
2,가르마펌
3,가일컷
4,가일펌
...,...
71,해병대머리
72,허쉬컷
73,호일펌
74,히메컷


In [19]:
df_styles['mecab'] = df_styles['style'].apply(mecab.morphs)
df_styles

,style,mecab
0,C컬펌,"[C, 컬, 펌]"
1,S컬펌,"[S, 컬, 펌]"
2,가르마펌,"[가르마, 펌]"
3,가일컷,"[가일, 컷]"
4,가일펌,"[가일, 펌]"
...,...,...
71,해병대머리,"[해병대, 머리]"
72,허쉬컷,"[허쉬, 컷]"
73,호일펌,"[호일, 펌]"
74,히메컷,[히메컷]


In [20]:
from collections import defaultdict
style_tag_dict = defaultdict(lambda: 'I-STY')
style_tag_dict[0] = 'B-STY'
style_tag_dict

defaultdict(<function __main__.<lambda>()>, {0: 'B-STY'})

In [21]:
def get_style_BIO_tag(morphs: list):
    return [style_tag_dict[i] for i in range(len(morphs))]

In [22]:
df_styles['BIO_tag'] = df_styles.mecab.apply(get_style_BIO_tag)
df_styles.head()

,style,mecab,BIO_tag
0,C컬펌,"[C, 컬, 펌]","[B-STY, I-STY, I-STY]"
1,S컬펌,"[S, 컬, 펌]","[B-STY, I-STY, I-STY]"
2,가르마펌,"[가르마, 펌]","[B-STY, I-STY]"
3,가일컷,"[가일, 컷]","[B-STY, I-STY]"
4,가일펌,"[가일, 펌]","[B-STY, I-STY]"


## 3) 장소

In [23]:
# 구
# 동
# 구 + 동
# 지하철역
# 위 네가지 케이스의 위치 단어을 만들자

In [24]:
shop_list = pd.read_csv('./output_data/HairshopList_2020-05-28.csv', index_col=0)
shop_list.head(2)

,store_id,store_name,tel,addrRoad,district,neighborhood,best_style,nearby_station,nearby_station_distance,review_count,coord_x,coord_y,collect_day
0,11547390,지모헤어 홍대점,02-3142-2562,서울특별시 마포구 와우산로23길 9,마포구,"서교동,서교동",붙임머리,"홍대입구역,상수역,광흥창역,합정역,서강대역","382,653,949,957,998",123,126.924334,37.553455,2020-05-22
1,11548559,다크헤어,02-2632-2060,서울특별시 영등포구 영등포로 133,영등포구,"당산동1가,당산1동","다운펌,가르마펌,아이롱펌","영등포구청역,문래역,영등포시장역","328,414,776",36,126.896539,37.521437,2020-05-22


In [25]:
import re

In [26]:
def remove_suffix(neighborhoods):
    # 길이가 3 이상인 경우만 뒷부분 제거
    tmp = [re.split('[0-9]+', neighborhood)[0] for neighborhood in neighborhoods.split(',')]
    tmp2=[]
    for val in tmp:
        if (len(val) > 2) & (val[-1] == '동'):
            tmp2.append(val[:-1])
        else:
            tmp2.append(val)
    
    # 순서 유지하며 중복제거
    input_dic = {}
    r_list = []
    for i, v in enumerate(tmp2):
        get_value = input_dic.get(v, None)
        if get_value == None:
            input_dic[v] = i
            r_list.append(v)
  
    return ','.join(r_list)

In [27]:
shop_list['district_no_suffix'] = shop_list.district.apply(lambda x: x[:-1] if x != '중구' else x)
shop_list['neighborhood_no_suffix'] = shop_list.neighborhood.apply(remove_suffix)
shop_list.head(10)

,store_id,store_name,tel,addrRoad,district,neighborhood,best_style,nearby_station,nearby_station_distance,review_count,coord_x,coord_y,collect_day,district_no_suffix,neighborhood_no_suffix
0,11547390,지모헤어 홍대점,02-3142-2562,서울특별시 마포구 와우산로23길 9,마포구,"서교동,서교동",붙임머리,"홍대입구역,상수역,광흥창역,합정역,서강대역","382,653,949,957,998",123,126.924334,37.553455,2020-05-22,마포,서교
1,11548559,다크헤어,02-2632-2060,서울특별시 영등포구 영등포로 133,영등포구,"당산동1가,당산1동","다운펌,가르마펌,아이롱펌","영등포구청역,문래역,영등포시장역","328,414,776",36,126.896539,37.521437,2020-05-22,영등포,당산
2,11561757,라메종뷰티,02-512-3001,서울특별시 강남구 선릉로146길 39,강남구,"청담동,청담동","허쉬컷,볼륨매직,루즈펌","강남구청역,압구정로데오역,청담역","527,670,984",26,127.042968,37.521781,2020-05-22,강남,청담
3,11567686,자쓰리헤어 구산점,02-313-8888,서울특별시 은평구 서오릉로 139,은평구,"역촌동,역촌동","투블럭,웨이브펌,샤기","구산역,역촌역,연신내역","47,699,890",183,126.917467,37.610850,2020-05-22,은평,역촌
4,11568230,뮤사이 살롱드점,1544-7442,서울특별시 강남구 선릉로157길 23-4,강남구,"신사동,압구정동",NaN,"압구정로데오역,압구정역","310,811",591,127.037667,37.525974,2020-05-22,강남,"신사,압구정"
5,11574385,준오헤어 이대1호점,02-392-6050,서울특별시 서대문구 이화여대길 28,서대문구,"대현동,신촌동",가르마펌,"이대역,신촌역,아현역","151,379,907",120,126.945910,37.558112,2020-05-22,서대문,"대현,신촌"
6,11578424,SM프로헤어 본점,02-469-5296,서울특별시 광진구 능동로13길 10,광진구,"화양동,화양동",가르마펌,"건대입구역,어린이대공원역","199,657",84,127.071378,37.542649,2020-05-22,광진,화양
7,11594334,자쓰리헤어 이대본점,02-312-6677,서울특별시 서대문구 이화여대8길 10,서대문구,"대현동,신촌동",NaN,"이대역,신촌역,아현역","207,387,881",161,126.946295,37.558678,2020-05-22,서대문,"대현,신촌"
8,11636317,리안헤어 대림역점,02-848-8401,서울특별시 영등포구 도림로31길 4,영등포구,"대림동,대림3동",NaN,"대림역,남구로역,구로디지털단지역","78,939,957",478,126.895873,37.492649,2020-05-22,영등포,대림
9,11659960,클레오파트라미용실,02-353-6993,서울특별시 서대문구 통일로 405,서대문구,"홍제동,홍제1동",볼륨매직,"홍제역,무악재역","422,453",76,126.947077,37.585784,2020-05-22,서대문,홍제


In [28]:
print(len(shop_list.columns))
shop_list = shop_list[['store_id', 'store_name', 'tel'
                       , 'addrRoad', 'district', 'district_no_suffix', 'neighborhood', 'neighborhood_no_suffix'
                       , 'best_style', 'nearby_station', 'nearby_station_distance'
                      , 'review_count'
                      , 'coord_x', 'coord_y'
                      , 'collect_day']]
print(len(shop_list.columns))

15
15


In [29]:
shop_list.head(2)

,store_id,store_name,tel,addrRoad,district,district_no_suffix,neighborhood,neighborhood_no_suffix,best_style,nearby_station,nearby_station_distance,review_count,coord_x,coord_y,collect_day
0,11547390,지모헤어 홍대점,02-3142-2562,서울특별시 마포구 와우산로23길 9,마포구,마포,"서교동,서교동",서교,붙임머리,"홍대입구역,상수역,광흥창역,합정역,서강대역","382,653,949,957,998",123,126.924334,37.553455,2020-05-22
1,11548559,다크헤어,02-2632-2060,서울특별시 영등포구 영등포로 133,영등포구,영등포,"당산동1가,당산1동",당산,"다운펌,가르마펌,아이롱펌","영등포구청역,문래역,영등포시장역","328,414,776",36,126.896539,37.521437,2020-05-22


In [30]:
#shop_list.to_csv('./output_data/HairshopList_2020-05-29.csv', sep=',', encoding='utf-8-sig')

In [31]:
# 서울시 전체 구 개수 : 25
district1 = shop_list.district.unique().tolist()
district2 = shop_list.district_no_suffix.unique().tolist()
print('구 개수:', len(district1), len(district2))

districts = list(set(district1 + district2))
print('구 합산 개수 :', len(districts))

None in districts

구 개수: 25 25
구 합산 개수 : 49


False

In [32]:
neighborhood1 = shop_list.neighborhood.tolist()
neighborhood1 = ','.join(neighborhood1)
neighborhood1 = list(set(neighborhood1.split(',')))

neighborhood2 = shop_list.neighborhood_no_suffix.tolist()
neighborhood2 = ','.join(neighborhood2)
neighborhood2 = list(set(neighborhood2.split(',')))
print('동 개수 :', len(neighborhood1), len(neighborhood2))

neighborhoods = list(set(neighborhood1 + neighborhood2))
print('동 합산 개수 :', len(neighborhoods))

None in neighborhoods

동 개수 : 565 279
동 합산 개수 : 832


False

In [33]:
district_neighborhood = []
for dis_val in districts:
    for neigh_val in neighborhoods:
        district_neighborhood.append(f'{dis_val} {neigh_val}')

print('구+동 개수:', len(district_neighborhood))

None in district_neighborhood

구+동 개수: 40768


False

In [34]:
# 얘는 서울 전체 지하철역 수 인데 
# 이걸 굳이 다 쓸 필요는 없을 것 같다.
seoul_station = pd.read_csv('./input_data/서울시 지하철역 정보 검색 (역명).csv', encoding='ms949')
len(set(seoul_station.전철역명.tolist()))

614

In [35]:
locations = list(set(districts + neighborhoods + district_neighborhood))
locations.sort()
#print('위치 단어 총 개수 :', len(locations))

### 3-1) 구, 동

In [36]:
df_locations = DataFrame({'location' : locations})
df_locations

,location
0,가락
1,가락1동
2,가락2동
3,가락동
4,가락본
...,...
41636,휘경1동
41637,휘경2동
41638,휘경동
41639,흑석


In [37]:
df_locations['mecab'] = df_locations.location.apply(mecab.morphs)
df_locations

,location,mecab
0,가락,[가락]
1,가락1동,"[가락, 1, 동]"
2,가락2동,"[가락, 2, 동]"
3,가락동,[가락동]
4,가락본,[가락본]
...,...,...
41636,휘경1동,"[휘경, 1, 동]"
41637,휘경2동,"[휘경, 2, 동]"
41638,휘경동,[휘경동]
41639,흑석,[흑석]


In [38]:
from collections import defaultdict
loc_tag_dict = defaultdict(lambda: 'I-LOC')
loc_tag_dict[0] = 'B-LOC'
loc_tag_dict

defaultdict(<function __main__.<lambda>()>, {0: 'B-LOC'})

In [39]:
def get_loc_BIO_tag(morphs: list):
    return [loc_tag_dict[i] for i in range(len(morphs))]

In [40]:
df_locations['BIO_tag'] = df_locations.mecab.apply(get_loc_BIO_tag)
df_locations

,location,mecab,BIO_tag
0,가락,[가락],[B-LOC]
1,가락1동,"[가락, 1, 동]","[B-LOC, I-LOC, I-LOC]"
2,가락2동,"[가락, 2, 동]","[B-LOC, I-LOC, I-LOC]"
3,가락동,[가락동],[B-LOC]
4,가락본,[가락본],[B-LOC]
...,...,...,...
41636,휘경1동,"[휘경, 1, 동]","[B-LOC, I-LOC, I-LOC]"
41637,휘경2동,"[휘경, 2, 동]","[B-LOC, I-LOC, I-LOC]"
41638,휘경동,[휘경동],[B-LOC]
41639,흑석,[흑석],[B-LOC]


In [41]:
stations = shop_list[shop_list.nearby_station.notnull()].nearby_station.tolist()
stations = ','.join(stations)
stations = list(set(stations.split(',')))
print('지하철역 개수 :', len(stations))

None in stations

지하철역 개수 : 295


False

### 3-2) 지하철역

In [42]:
df_stations = DataFrame({'station' : stations})
df_stations

,station
0,등촌역
1,용산역
2,이대역
3,하계역
4,흑석역
...,...
290,오목교역
291,한티역
292,수락산역
293,보문역


In [43]:
df_stations['mecab'] = df_stations.station.apply(mecab.morphs)
df_stations

,station,mecab
0,등촌역,[등촌역]
1,용산역,[용산역]
2,이대역,"[이, 대역]"
3,하계역,[하계역]
4,흑석역,"[흑, 석역]"
...,...,...
290,오목교역,"[오목, 교역]"
291,한티역,[한티역]
292,수락산역,[수락산역]
293,보문역,[보문역]


In [44]:
from collections import defaultdict
station_tag_dict = defaultdict(lambda: 'I-SBW')
station_tag_dict[0] = 'B-SBW'
station_tag_dict

defaultdict(<function __main__.<lambda>()>, {0: 'B-SBW'})

In [45]:
def get_station_BIO_tag(morphs: list):
    return [station_tag_dict[i] for i in range(len(morphs))]

In [46]:
df_stations['BIO_tag'] = df_stations.mecab.apply(get_station_BIO_tag)
df_stations

,station,mecab,BIO_tag
0,등촌역,[등촌역],[B-SBW]
1,용산역,[용산역],[B-SBW]
2,이대역,"[이, 대역]","[B-SBW, I-SBW]"
3,하계역,[하계역],[B-SBW]
4,흑석역,"[흑, 석역]","[B-SBW, I-SBW]"
...,...,...,...
290,오목교역,"[오목, 교역]","[B-SBW, I-SBW]"
291,한티역,[한티역],[B-SBW]
292,수락산역,[수락산역],[B-SBW]
293,보문역,[보문역],[B-SBW]


## 4) 업소명

In [47]:
shops = shop_list.store_name.tolist()
shops

['지모헤어 홍대점',
 '다크헤어',
 '라메종뷰티',
 '자쓰리헤어 구산점',
 '뮤사이 살롱드점',
 '준오헤어 이대1호점',
 'SM프로헤어 본점',
 '자쓰리헤어 이대본점',
 '리안헤어 대림역점',
 '클레오파트라미용실',
 '젠헤어 이대점',
 '마지크',
 '진우헤어 연희점',
 'HMTT미용실',
 '이영미 헤어',
 '다다헤어',
 '와이미용실',
 '박승철헤어스투디오 관악롯데백화점',
 '박승철헤어스투디오 문정점',
 '박승철헤어스투디오 압구정로데오점',
 '준오헤어 강남2호점',
 '준오헤어 강남1호점',
 '준오헤어 대치점',
 '준오헤어 돈암2호점',
 '나이스가이 여의도점',
 '이가자헤어비스 홍대점',
 '쟈끄데상쥬 신천점',
 '하이크라스 미용실',
 '에이컨셉 청담 본점',
 '라비에벨헤어 이대본점',
 '이철헤어커커 명동1호점',
 '명동헤어뱅크',
 '헤어바이반룬하우트 연신내점',
 '류영헤어투비',
 '박준뷰티랩 강남역점',
 '쟝피엘헤어 서울대입구역점',
 '이경민포레 청담본점',
 '나건아헤어',
 '제이앤에스헤어',
 '에반스타일',
 '라보노헤어 1호점',
 '이철헤어커커 건대스타시티점',
 '민스헤어 이대점',
 '쿠헤어',
 '아이디헤어 잠원점',
 '아이디헤어 반포1호점',
 '아뜰리에7',
 '라리엔 명동점',
 '프랑크프로보 명동',
 '도쿄랜드',
 '헤어캐슬24시붙임머리전문점 논현지점',
 '버블스토리 NC백화점불광점',
 '마니아미용실',
 '스타일봄헤어살롱 1호점',
 '세븐스트릿',
 '이지연더스타일',
 '리안헤어 신세계남대문점',
 '헤어닥터스',
 '헤어에스',
 '이가자헤어비스 은평이마트점',
 '순시키헤어 밤헤드',
 '리브헤어',
 '스타일플로어 임진옥',
 '파비엔에이치',
 '킹스턴 맨즈헤어',
 '양리헤어',
 '레트로나인',
 '엔비헤어살롱',
 '마샬뷰티살롱 논현점',
 '컬리헤어',
 '쎄봉',
 '리안헤어 잠실역점',
 '아리아헤어',
 '버블스타일',
 '펑크샬롬',
 '제

In [48]:
df_hairshops = DataFrame({'store_name' : shops})
df_hairshops

,store_name
0,지모헤어 홍대점
1,다크헤어
2,라메종뷰티
3,자쓰리헤어 구산점
4,뮤사이 살롱드점
...,...
2677,토브헤어 숙대점
2678,인더살롱 수유본점
2679,나우헤어
2680,헤어디자인모습 두번째점


In [49]:
df_hairshops['mecab'] = df_hairshops.store_name.apply(mecab.morphs)
df_hairshops

,store_name,mecab
0,지모헤어 홍대점,"[지모, 헤어, 홍대, 점]"
1,다크헤어,"[다크, 헤어]"
2,라메종뷰티,"[라메, 종, 뷰티]"
3,자쓰리헤어 구산점,"[자, 쓰리, 헤어, 구산, 점]"
4,뮤사이 살롱드점,"[뮤, 사이, 살, 롱드, 점]"
...,...,...
2677,토브헤어 숙대점,"[토브, 헤어, 숙대, 점]"
2678,인더살롱 수유본점,"[인더, 살롱, 수유, 본점]"
2679,나우헤어,"[나우, 헤어]"
2680,헤어디자인모습 두번째점,"[헤어, 디자인, 모습, 두, 번, 째, 점]"


In [50]:
from collections import defaultdict
storename_tag_dict = defaultdict(lambda: 'I-ORG')
storename_tag_dict[0] = 'B-ORG'
storename_tag_dict

defaultdict(<function __main__.<lambda>()>, {0: 'B-ORG'})

In [51]:
def get_storename_BIO_tag(morphs: list):
    return [storename_tag_dict[i] for i in range(len(morphs))]

In [52]:
df_hairshops['BIO_tag'] = df_hairshops.mecab.apply(get_storename_BIO_tag)
df_hairshops

,store_name,mecab,BIO_tag
0,지모헤어 홍대점,"[지모, 헤어, 홍대, 점]","[B-ORG, I-ORG, I-ORG, I-ORG]"
1,다크헤어,"[다크, 헤어]","[B-ORG, I-ORG]"
2,라메종뷰티,"[라메, 종, 뷰티]","[B-ORG, I-ORG, I-ORG]"
3,자쓰리헤어 구산점,"[자, 쓰리, 헤어, 구산, 점]","[B-ORG, I-ORG, I-ORG, I-ORG, I-ORG]"
4,뮤사이 살롱드점,"[뮤, 사이, 살, 롱드, 점]","[B-ORG, I-ORG, I-ORG, I-ORG, I-ORG]"
...,...,...,...
2677,토브헤어 숙대점,"[토브, 헤어, 숙대, 점]","[B-ORG, I-ORG, I-ORG, I-ORG]"
2678,인더살롱 수유본점,"[인더, 살롱, 수유, 본점]","[B-ORG, I-ORG, I-ORG, I-ORG]"
2679,나우헤어,"[나우, 헤어]","[B-ORG, I-ORG]"
2680,헤어디자인모습 두번째점,"[헤어, 디자인, 모습, 두, 번, 째, 점]","[B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG]"


## 5) 어미

In [53]:
import pickle

with open('./output_data/ending_words_re.pickle', 'rb') as f:
    ending_words_r = pickle.load(f)

with open('./output_data/ending_words_que_loc.pickle', 'rb') as f:
    ending_words_q_loc = pickle.load(f)
    
with open('./output_data/ending_words_que_price.pickle', 'rb') as f:
    ending_words_q_price = pickle.load(f)
    
with open('./output_data/ending_words_que_review.pickle', 'rb') as f:
    ending_words_q_review = pickle.load(f)

In [54]:
df_ending_words_r = DataFrame({'ending_words_r': ending_words_r})
df_ending_words_r

,ending_words_r
0,추천해줄만한 데 물어볼게요
1,훌륭한 데 있죠
2,실패한적 없는 데 어디 없나요
3,실패한적 없는 헤어샵 있으신지요
4,지리는 곳 어디였죠
...,...
12461,좋은 헤어샵 어디있나요
12462,잘하는 데 부탁해
12463,추천가능한 헤어샵 궁금한데요
12464,추천할 데 궁금한데요


In [55]:
df_ending_words_q_loc = DataFrame({'ending_words_q_loc': ending_words_q_loc})
df_ending_words_q_loc

,ending_words_q_loc
0,상세주소 알까
1,자세한 위치 알려줘
2,주소 알려줘요
3,자세한 위치 물어봐
4,장소 아냐
...,...
454,상세주소 어디 위치해있죠
455,자세한 장소 어디에있어요
456,어디냐
457,주소 물어봐


In [56]:
df_ending_words_q_price = DataFrame({'ending_words_q_price': ending_words_q_price})
df_ending_words_q_price

,ending_words_q_price
0,비용 알아
1,값 아세요
2,액수 물어볼려고요
3,비용 알아요
4,하는 가격 얼마입니까?
...,...
490,어떻죠?
491,값 질문합니다
492,값 부탁드려요
493,질문합니다


In [57]:
df_ending_words_q_review = DataFrame({'ending_words_q_review': ending_words_q_review})
df_ending_words_q_review

,ending_words_q_review
0,평 뭐죠
1,광고없는리뷰 여쭤볼라고
2,가본 사람들 평가 없습니까
3,솔직반응 아세요
4,반응 뭘까요
...,...
3532,사람들 후기 알려줄래
3533,솔직한 평가 가르쳐주세요
3534,솔직평가 질문드려요
3535,진짜리뷰 찾아줘


In [58]:
# 한글만 남기기
def get_only_korean(string):
    return re.sub('[^ ㄱ-ㅎ가-힣]+', '', string)

In [59]:
# 혹시 모를 한글 제거
df_ending_words_r['ending_words_r'] = df_ending_words_r.ending_words_r.apply(get_only_korean)
df_ending_words_q_loc['ending_words_q_loc'] = df_ending_words_q_loc.ending_words_q_loc.apply(get_only_korean)
df_ending_words_q_price['ending_words_q_price'] = df_ending_words_q_price.ending_words_q_price.apply(get_only_korean)
df_ending_words_q_review['ending_words_q_review'] = df_ending_words_q_review.ending_words_q_review.apply(get_only_korean)

In [60]:
# 추천 어미 형태소 분석
df_ending_words_r['mecab'] = df_ending_words_r.ending_words_r.apply(mecab.morphs)
df_ending_words_r.head(2)

,ending_words_r,mecab
0,추천해줄만한 데 물어볼게요,"[추천, 해, 줄, 만, 한, 데, 물어볼, 게요]"
1,훌륭한 데 있죠,"[훌륭, 한, 데, 있, 죠]"


In [61]:
# 위치 문의 어미 형태소 분석
df_ending_words_q_loc['mecab'] = df_ending_words_q_loc.ending_words_q_loc.apply(mecab.morphs)
df_ending_words_q_loc.head(2)

,ending_words_q_loc,mecab
0,상세주소 알까,"[상세, 주소, 알, 까]"
1,자세한 위치 알려줘,"[자세, 한, 위치, 알려줘]"


In [62]:
# 가격 문의 어미 형태소 분석
df_ending_words_q_price['mecab'] = df_ending_words_q_price.ending_words_q_price.apply(mecab.morphs)
df_ending_words_q_price.head(2)

,ending_words_q_price,mecab
0,비용 알아,"[비용, 알, 아]"
1,값 아세요,"[값, 아세요]"


In [63]:
# 리뷰 문의 어미 형태소 분석
df_ending_words_q_review['mecab'] = df_ending_words_q_review.ending_words_q_review.apply(mecab.morphs)
df_ending_words_q_review.head(2)

,ending_words_q_review,mecab
0,평 뭐죠,"[평, 뭐, 죠]"
1,광고없는리뷰 여쭤볼라고,"[광고, 없, 는, 리뷰, 여쭤, 볼라고]"


In [64]:
from collections import defaultdict
out_of_tag_dict = defaultdict(lambda: 'O')
out_of_tag_dict

defaultdict(<function __main__.<lambda>()>, {})

In [65]:
def get_out_of_BIO_tag(morphs: list):
    return [out_of_tag_dict[i] for i in range(len(morphs))]

In [66]:
# 추천 어미 BIO 태깅
df_ending_words_r['BIO_tag'] = df_ending_words_r.mecab.apply(get_out_of_BIO_tag)
df_ending_words_r.head(2)

,ending_words_r,mecab,BIO_tag
0,추천해줄만한 데 물어볼게요,"[추천, 해, 줄, 만, 한, 데, 물어볼, 게요]","[O, O, O, O, O, O, O, O]"
1,훌륭한 데 있죠,"[훌륭, 한, 데, 있, 죠]","[O, O, O, O, O]"


In [67]:
# 위치 문의 어미 BIO 태깅
df_ending_words_q_loc['BIO_tag'] = df_ending_words_q_loc.mecab.apply(get_out_of_BIO_tag)
df_ending_words_q_loc.head(2)

,ending_words_q_loc,mecab,BIO_tag
0,상세주소 알까,"[상세, 주소, 알, 까]","[O, O, O, O]"
1,자세한 위치 알려줘,"[자세, 한, 위치, 알려줘]","[O, O, O, O]"


In [68]:
# 가격 문의 어미 BIO 태깅
df_ending_words_q_price['BIO_tag'] = df_ending_words_q_price.mecab.apply(get_out_of_BIO_tag)
df_ending_words_q_price.head(2)

,ending_words_q_price,mecab,BIO_tag
0,비용 알아,"[비용, 알, 아]","[O, O, O]"
1,값 아세요,"[값, 아세요]","[O, O]"


In [69]:
# 리뷰 문의 어미 BIO 태깅
df_ending_words_q_review['BIO_tag'] = df_ending_words_q_review.mecab.apply(get_out_of_BIO_tag)
df_ending_words_q_review.head(2)

,ending_words_q_review,mecab,BIO_tag
0,평 뭐죠,"[평, 뭐, 죠]","[O, O, O]"
1,광고없는리뷰 여쭤볼라고,"[광고, 없, 는, 리뷰, 여쭤, 볼라고]","[O, O, O, O, O, O]"


---
# 2. 질문 말뭉치 만들기


### 완성된 BIO tagging 말뭉치
- 1) 형용사
    - 1-1) 전체형용사
    - 1-2) 위치형용사
- 2) 스타일
- 3) 장소
    - 3-1) 구, 동
    - 3-2) 지하철역
- 4) 업소명
- 5) 어미
    - 추천요청어미
    - 위치문의어미
    - 가격문의어미
    - 리뷰문의어미

In [70]:
print('전체형용사 :', len(df_adjs_all))
print('위치형용사 :', len(df_adjs_loc))
print('스타일 :', len(df_styles))
print('장소 :', len(df_locations))
print('지하철역 :', len(df_stations))
print('업소명 :', len(df_hairshops))
print('추천요청어미 :', len(df_ending_words_r))
print('위치문의어미 :', len(df_ending_words_q_loc))
print('가격문의어미 :', len(df_ending_words_q_price))
print('리뷰문의어미 :', len(df_ending_words_q_review))


전체형용사 : 208
위치형용사 : 22
스타일 : 76
장소 : 41641
지하철역 : 295
업소명 : 2682
추천요청어미 : 12466
위치문의어미 : 459
가격문의어미 : 495
리뷰문의어미 : 3537


### 1) 추천요청 질문 말뭉치
- 1-1) 스타일 + 추천요청어미
- 1-2) 스타일 + 형용사 + 추천요청어미
- 1-3) 스타일 + 형용사 + 장소 + 추천요청어미
- 1-4) 스타일 + 장소 + 형용사 + 추천요청어미
- 1-5) 스타일 + 지하철역 + 위치형용사 + 추천요청어미
- 1-6) 형용사 + 추천요청어미
- 1-7) 형용사 + 스타일 + 추천요청어미
- 1-8) 형용사 + 형용사 + 추천요청어미
- 1-9) 형용사 + 스타일 + 장소 + 추천요청어미
- 1-10) 형용사 + 스타일 + 지하철역 + 위치형용사 + 추천요청어미
- 1-11) 장소 + 추천요청어미 
- 1-12) 장소 + 스타일 + 추천요청어미
- 1-13) 장소 + 형용사 + 추천요청어미
- 1-14) 장소 + 형용사 + 스타일 + 추천요청어미
- 1-15) 지하철역 + 위치형용사 + 추천요청어미
- 1-16) 지하철역 + 위치형용사 + 형용사 + 추천요청어미

### 2) 문의(위치, 가격, 리뷰) 질문 말뭉치
- 2-1) 업소명 + 위치문의어미
- 2-2) 업소명 + 가격문의어미
- 2-3) 업소명 + 리뷰문의어미
- 2-4) 업소명 + 스타일 + 리뷰문의어미
- 2-5) 장소 + 업소명 + 위치문의어미
- 2-6) 장소 + 업소명 + 가격문의어미
- 2-7) 장소 + 업소명 + 리뷰문의어미
- 2-8) 장소 + 업소명 + 스타일 + 리뷰문의어미
- 2-9) 지하철역 + 업소명 + 위치문의어미
- 2-10) 지하철역 + 업소명 + 가격문의어미
- 2-11) 지하철역 + 업소명 + 리뷰문의어미
- 2-12) 지하철역 + 업소명 + 스타일 + 리뷰문의어미 

In [71]:
# 1-1) 스타일 + 추천요청어미
r1 = len(df_styles) * len(df_ending_words_r)

# 1-2) 스타일 + 형용사 + 추천요청어미
r2 = len(df_styles) * len(df_adjs_all) * len(df_ending_words_r)

# 1-3) 스타일 + 형용사 + 장소 + 추천요청어미
r3 = len(df_styles) * len(df_adjs_all) * len(df_locations) * len(df_ending_words_r)

# 1-4) 스타일 + 장소 + 형용사 + 추천요청어미
r4 = len(df_styles) * len(df_locations) * len(df_adjs_all) * len(df_ending_words_r)

# 1-5) 스타일 + 지하철역 + 위치형용사 + 추천요청어미
r5 = len(df_styles) * len(df_stations) * len(df_adjs_loc) * len(df_ending_words_r)

# 1-6) 형용사 + 추천요청어미
r6 = len(df_adjs_all) * len(df_ending_words_r)

# 1-7) 형용사 + 스타일 + 추천요청어미
r7 = len(df_adjs_all) * len(df_styles) * len(df_ending_words_r)

# 1-8) 형용사 + 형용사 + 추천요청어미
r8 = len(df_adjs_all) * len(df_adjs_all) * len(df_ending_words_r)

# 1-9) 형용사 + 스타일 + 장소 + 추천요청어미
r9 = len(df_adjs_all) * len(df_styles) * len(df_locations) * len(df_ending_words_r)

# 1-10) 형용사 + 스타일 + 지하철역 + 위치형용사 + 추천요청어미
r10 = len(df_adjs_all) * len(df_styles) * len(df_stations) * len(df_adjs_loc) * len(df_ending_words_r)

# 1-11) 장소 + 추천요청어미 
r11 = len(df_locations) * len(df_ending_words_r)

# 1-12) 장소 + 스타일 + 추천요청어미
r12 = len(df_locations) * len(df_styles) * len(df_ending_words_r)

# 1-13) 장소 + 형용사 + 추천요청어미
r13 = len(df_locations) * len(df_adjs_all) * len(df_ending_words_r)

# 1-14) 장소 + 형용사 + 스타일 + 추천요청어미
r14 = len(df_locations) * len(df_adjs_all) * len(df_styles) * len(df_ending_words_r)

# 1-15) 지하철역 + 위치형용사 + 추천요청어미
r15 = len(df_stations) * len(df_adjs_loc) * len(df_ending_words_r)

# 1-16) 지하철역 + 위치형용사 + 형용사 + 추천요청어미
r16 = len(df_stations) * len(df_adjs_loc) * len(df_adjs_all) * len(df_ending_words_r)

print('r1 :', r1)
print('r2 :', r2)
print('r3 :', r3)
print('r4 :', r4)
print('r5 :', r5)
print('r6 :', r6)
print('r7 :', r7)
print('r8 :', r8)
print('r9 :', r9)
print('r10 :', r10)
print('r11 :', r11)
print('r12 :', r12)
print('r13 :', r13)
print('r14 :', r14)
print('r15 :', r15)
print('r16 :', r16)
print('r total :', r1 + r2 + r3 + r4 + r5 + r6 + r7 + r8 + r9 + r10 + r11 + r12 + r13 + r14 + r15 + r16)

r1 : 947416
r2 : 197062528
r3 : 8205880728448
r4 : 8205880728448
r5 : 6148729840
r6 : 2592928
r7 : 197062528
r8 : 539329024
r9 : 8205880728448
r10 : 1278935806720
r11 : 519096706
r12 : 39451349656
r13 : 107972114848
r14 : 8205880728448
r15 : 80904340
r16 : 16828102720
r total : 34274396013046


In [72]:
100/16

6.25

각 추천 질문당 6만2천5백개 씩 만들면 100만개 추천요청 질문 생성!

In [73]:
# 2-1) 업소명 + 위치문의어미
q1 = len(df_hairshops) * len(df_ending_words_q_loc)

# 2-2) 업소명 + 가격문의어미
q2 = len(df_hairshops) * len(df_ending_words_q_price)

# 2-3) 업소명 + 리뷰문의어미
q3 = len(df_hairshops) * len(df_ending_words_q_review)

# 2-4) 업소명 + 스타일 + 리뷰문의어미
q4 = len(df_hairshops) * len(df_styles) * len(df_ending_words_q_review)

# 2-5) 장소 + 업소명 + 위치문의어미
q5 = len(df_locations) * len(df_hairshops) * len(df_ending_words_q_loc)

# 2-6) 장소 + 업소명 + 가격문의어미
q6 = len(df_locations) * len(df_hairshops) * len(df_ending_words_q_price)

# 2-7) 장소 + 업소명 + 리뷰문의어미
q7 = len(df_locations) * len(df_hairshops) * len(df_ending_words_q_review)

# 2-8) 장소 + 업소명 + 스타일 + 리뷰문의어미
q8 = len(df_locations) * len(df_hairshops) * len(df_styles) * len(df_ending_words_q_review)

# 2-9) 지하철역 + 업소명 + 위치문의어미
q9 = len(df_stations) * len(df_hairshops) * len(df_ending_words_q_loc)

# 2-10) 지하철역 + 업소명 + 가격문의어미
q10 = len(df_stations) * len(df_hairshops) * len(df_ending_words_q_price)

# 2-11) 지하철역 + 업소명 + 리뷰문의어미 
q11 = len(df_stations) * len(df_hairshops) * len(df_ending_words_q_review)

# 2-12) 지하철역 + 업소명 + 스타일 + 리뷰문의어미 
q12 = len(df_stations) * len(df_hairshops) * len(df_styles) * len(df_ending_words_q_review)

print('q1 :', q1)
print('q2 :', q2)
print('q3 :', q3)
print('q4 :', q4)
print('q5 :', q5)
print('q6 :', q6)
print('q7 :', q7)
print('q8 :', q8)
print('q9 :', q9)
print('q10 :', q10)
print('q11 :', q11)
print('q12 :', q12)
print('q total :', q1 + q2 + q3 + q4 + q5 + q6 + q7 + q8 + q9 + q10 + q11 + q12)

q1 : 1231038
q2 : 1327590
q3 : 9486234
q4 : 720953784
q5 : 51261653358
q6 : 55282175190
q7 : 395016269994
q8 : 30021236519544
q9 : 363156210
q10 : 391639050
q11 : 2798439030
q12 : 212681366280
q total : 30739764217302


In [74]:
100 / 12

8.333333333333334

각 문의 질문당 8만 3천 3백 3십 3개씩 만들면 문의 질문 100만개 생성!

---
## 1) 추천요청 질문 생성

In [75]:
import random
from tqdm.notebook import tqdm

In [76]:
# 추천 질문을 저장
r_questions = []

# 추천 질문 형태소 분석 결과를 저장
r_mecabs = []

# 추천 질문 BIO 태그를 저장
r_tags = []

In [77]:
def make_sentences(q_type: str, dfs: list):
    
    idxes = list(range(len(dfs)))
    for i, df in enumerate(dfs):
        idxes[i] = random.randint(0, len(df)-1)
    
    rows = list(range(len(dfs)))
    for i, df in enumerate(dfs):
        rows[i] = df.iloc[idxes[i]]
    
    tmp_question = ''
    tmp_mecab = []
    tmp_tag = []
    for i, df in enumerate(dfs):
        if df.columns[0] == 'style':
            if q_type == 'r':
                tmp_question += rows[i][0] + ' 잘하는' + ' '
                tmp_mecab += rows[i][1] + ['잘', '하', '는']
                tmp_tag += rows[i][2] + ['O', 'O', 'O']
            else:
                tmp_question += rows[i][0] + ' '
                tmp_mecab += rows[i][1]
                tmp_tag += rows[i][2]
        elif df.columns[0] == 'ending_words_r':
            if '잘하는' in rows[i][0]:
                tmp_question = tmp_question[:-5]
                tmp_mecab = tmp_mecab[:-3]
                tmp_tag = tmp_tag[:-3]
            tmp_question += rows[i][0] + ' '
            tmp_mecab += rows[i][1]
            tmp_tag += rows[i][2]
        
        elif df.columns[0] == 'location':
            tmp_question += rows[i][0] + '에' + ' '
            tmp_mecab += rows[i][1] + ['에']
            tmp_tag += rows[i][2] + ['O']
        
        elif df.columns[0] == 'station':
            tmp_question += rows[i][0] + '에서' + ' '
            tmp_mecab += rows[i][1] + ['에서']
            tmp_tag += rows[i][2] + ['O']
        
        else:
            tmp_question += rows[i][0] + ' '
            tmp_mecab += rows[i][1]
            tmp_tag += rows[i][2]

    tmp_question =tmp_question.strip()
    
    return tmp_question, tmp_mecab, tmp_tag

In [78]:
# 1-1) 스타일 + 추천요청어미
print('1-1) 스타일 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_styles, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-2) 스타일 + 형용사 + 추천요청어미
print('1-2) 스타일 + 형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_styles, df_adjs_all, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-3) 스타일 + 형용사 + 장소 + 추천요청어미
print('1-3) 스타일 + 형용사 + 장소 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_styles, df_adjs_all, df_locations, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-4) 스타일 + 장소 + 형용사 + 추천요청어미
print('1-4) 스타일 + 장소 + 형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_styles, df_locations, df_adjs_all, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-5) 스타일 + 지하철역 + 위치형용사 + 추천요청어미
print('1-5) 스타일 + 지하철역 + 위치형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_styles, df_stations, df_adjs_loc, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-6) 형용사 + 추천요청어미
print('1-6) 형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_adjs_all, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-7) 형용사 + 스타일 + 추천요청어미
print('1-7) 형용사 + 스타일 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_adjs_all, df_styles, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-8) 형용사 + 형용사 + 추천요청어미
print('1-8) 형용사 + 형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_adjs_all, df_adjs_all, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-9) 형용사 + 스타일 + 장소 + 추천요청어미
print('1-9) 형용사 + 스타일 + 장소 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_adjs_all, df_styles, df_locations, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-10) 형용사 + 스타일 + 지하철역 + 위치형용사 + 추천요청어미
print('1-10) 형용사 + 스타일 + 지하철역 + 위치형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_adjs_all, df_styles, df_stations, df_adjs_loc, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-11) 장소 + 추천요청어미 
print('1-11) 장소 + 추천요청어미 ')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_locations, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-12) 장소 + 스타일 + 추천요청어미
print('1-12) 장소 + 스타일 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_locations, df_styles, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-13) 장소 + 형용사 + 추천요청어미
print('1-13) 장소 + 형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_locations, df_adjs_all, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-14) 장소 + 형용사 + 스타일 + 추천요청어미
print('1-14) 장소 + 형용사 + 스타일 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_locations, df_adjs_all, df_styles, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-15) 지하철역 + 위치형용사 + 추천요청어미
print('1-15) 지하철역 + 위치형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_stations, df_adjs_loc, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

# 1-16) 지하철역 + 위치형용사 + 형용사 + 추천요청어미
print('1-16) 지하철역 + 위치형용사 + 형용사 + 추천요청어미')
for i in tqdm(range(62500)):
    
    result = make_sentences('r', [df_stations, df_adjs_loc, df_adjs_all, df_ending_words_r])
    
    r_questions.append(result[0])
    r_mecabs.append(result[1])
    r_tags.append(result[2])

print(r_questions[-1])
print(r_mecabs[-1])
print(r_tags[-1])
print()

1-1) 스타일 + 추천요청어미



페미닌펌 잘하는 실패하지않는 데 뭐였어요
['페미', '닌', '펌', '잘', '하', '는', '실패', '하', '지', '않', '는', '데', '뭐', '였어요']
['B-STY', 'I-STY', 'I-STY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

1-2) 스타일 + 형용사 + 추천요청어미



트위스트펌 잘하는 우울하지 않은 예쁘게 하는 미용실 뭐지요
['트위스트', '펌', '잘', '하', '는', '우울', '하', '지', '않', '은', '예쁘', '게', '하', '는', '미용실', '뭐', '지요']
['B-STY', 'I-STY', 'O', 'O', 'O', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

1-3) 스타일 + 형용사 + 장소 + 추천요청어미



언발란스컷 잘하는 파격적인 가격의 서대문구 충무로에 추천할 데 어디일까요
['언', '발란스', '컷', '잘', '하', '는', '파격', '적', '인', '가격', '의', '서대문구', '충무로', '에', '추천', '할', '데', '어디', '일까요']
['B-STY', 'I-STY', 'I-STY', 'O', 'O', 'O', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'B-LOC', 'I-LOC', 'O', 'O', 'O', 'O', 'O', 'O']

1-4) 스타일 + 장소 + 형용사 + 추천요청어미



내츄럴펌 잘하는 관악 중림에 깨끗한 잘해주는 미용실 있어예
['내츄럴', '펌', '잘', '하', '는', '관악', '중림', '에', '깨끗', '한', '잘', '해', '주', '는', '미용실', '있', '어', '예']
['B-STY', 'I-STY', 'O', 'O', 'O', 'B-LOC', 'I-LOC', 'O', 'B-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

1-5) 스타일 + 지하철역 + 위치형용사 + 추천요청어미


...


송파 사당3동에 스왓컷 잘하는 좋은 곳 알려줄 수 있나요
['송파', '사당', '3', '동', '에', '스', '왓', '컷', '잘', '하', '는', '좋', '은', '곳', '알려', '줄', '수', '있', '나요']
['B-LOC', 'I-LOC', 'I-LOC', 'I-LOC', 'O', 'B-STY', 'I-STY', 'I-STY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

1-13) 장소 + 형용사 + 추천요청어미



영등포구 역촌에 아늑한 잘해주는 곳 알려줘요
['영등포구', '역촌', '에', '아늑', '한', '잘', '해', '주', '는', '곳', '알려', '줘요']
['B-LOC', 'I-LOC', 'O', 'B-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

1-14) 장소 + 형용사 + 스타일 + 추천요청어미



서대문구 남영에 두상에 맞는 매직펌 잘하는 데려오고 싶은 곳 알려주시겠습니까
['서대문구', '남영', '에', '두상', '에', '맞', '는', '매직', '펌', '잘', '하', '는', '데려오', '고', '싶', '은', '곳', '알려', '주', '시', '겠', '습니까']
['B-LOC', 'I-LOC', 'O', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'B-STY', 'I-STY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']

1-15) 지하철역 + 위치형용사 + 추천요청어미



수유역에서 걸어갈만한 거리의 지리는 데 여쭤보려구요
['수유역', '에서', '걸어갈', '만', '한', '거리의', '지리', '는', '데', '여쭤', '보', '려구요']
['B-SBW', 'O', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O', 'O', 'O']

1-16) 지하철역 + 위치형용사 + 형용사 + 추천요청어미



신대방삼거리역에서 근처의 덜 뚱뚱해보이는 지리는 헤어샵 어디인데요
['신대방삼거리역', '에서', '근', '처의', '덜', '뚱뚱', '해', '보이', '는', '지리', '는', '헤어', '샵', '어디', '인데요']
['B-SBW', 'O', 'B-ADJ', 'I-ADJ', 'B-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'I-ADJ', 'O', 'O', 'O', 'O', 'O', 'O']



In [79]:
len(r_questions), len(r_mecabs), len(r_tags)

(1000000, 1000000, 1000000)

In [80]:
# 문의 질문을 저장
q_questions = []

# 문의 질문 형태소 분석 결과를 저장
q_mecabs = []

# 문의 질문 BIO 태그를 저장
q_tags = []

In [81]:
# 2-1) 업소명 + 위치문의어미
print('2-1) 업소명 + 위치문의어미')
for i in tqdm(range(83333)):
    
    result = make_sentences('q', [df_hairshops, df_ending_words_q_loc])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-2) 업소명 + 가격문의어미
print('2-2) 업소명 + 가격문의어미')
for i in tqdm(range(83333)):
    
    result = make_sentences('q', [df_hairshops, df_ending_words_q_price])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-3) 업소명 + 리뷰문의어미
print('2-3) 업소명 + 리뷰문의어미')
for i in tqdm(range(83333)):
    
    result = make_sentences('q', [df_hairshops, df_ending_words_q_review])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-4) 업소명 + 스타일 + 리뷰문의어미
print('2-4) 업소명 + 스타일 + 리뷰문의어미')
for i in tqdm(range(83333)):
    
    result = make_sentences('q', [df_hairshops, df_styles, df_ending_words_q_review])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-5) 장소 + 업소명 + 위치문의어미
print('2-5) 장소 + 업소명 + 위치문의어미')
for i in tqdm(range(83333)):
    
    result = make_sentences('q', [df_locations, df_hairshops, df_ending_words_q_loc])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-6) 장소 + 업소명 + 가격문의어미
print('2-6) 장소 + 업소명 + 가격문의어미')
for i in tqdm(range(83333)):
    
    result = make_sentences('q', [df_locations, df_hairshops, df_ending_words_q_price])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-7) 장소 + 업소명 + 리뷰문의어미
print('2-7) 장소 + 업소명 + 리뷰문의어미')
for i in tqdm(range(83333)):
    
    result = make_sentences('q', [df_locations, df_hairshops, df_ending_words_q_review])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-8) 장소 + 업소명 + 스타일 + 리뷰문의어미
print('2-8) 장소 + 업소명 + 스타일 + 리뷰문의어미')
for i in tqdm(range(83333)):
    
    result = make_sentences('q', [df_locations, df_hairshops, df_styles, df_ending_words_q_review])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-9) 지하철역 + 업소명 + 위치문의어미
print('2-9) 지하철역 + 업소명 + 위치문의어미')
for i in tqdm(range(83334)):
    
    result = make_sentences('q', [df_stations, df_hairshops, df_ending_words_q_loc])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-10) 지하철역 + 업소명 + 가격문의어미
print('2-10) 지하철역 + 업소명 + 가격문의어미')
for i in tqdm(range(83334)):
    
    result = make_sentences('q', [df_stations, df_hairshops, df_ending_words_q_price])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-11) 지하철역 + 업소명 + 리뷰문의어미
print('2-11) 지하철역 + 업소명 + 리뷰문의어미')
for i in tqdm(range(83334)):
    
    result = make_sentences('q', [df_stations, df_hairshops, df_ending_words_q_review])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

# 2-12) 지하철역 + 업소명 + 스타일 + 리뷰문의어미
print('2-12) 지하철역 + 업소명 + 스타일 + 리뷰문의어미')
for i in tqdm(range(83334)):
    
    result = make_sentences('q', [df_stations, df_hairshops, df_styles, df_ending_words_q_review])
    
    q_questions.append(result[0])
    q_mecabs.append(result[1])
    q_tags.append(result[2])

print(q_questions[-1])
print(q_mecabs[-1])
print(q_tags[-1])
print('\n')

2-1) 업소명 + 위치문의어미



이철헤어커커 건대스타시티점 위치 어디입니까
['이철', '헤어', '커', '커', '건대', '스타', '시티', '점', '위치', '어디', '입니까']
['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O']


2-2) 업소명 + 가격문의어미



가꿈헤어드림 명동점 요금 아시나요
['가꿈', '헤어', '드림', '명', '동점', '요금', '아시나', '요']
['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O']


2-3) 업소명 + 리뷰문의어미



가디 헤어샵 진짜반응 찾아주세요
['가디', '헤어', '샵', '진짜', '반응', '찾', '아', '주', '세요']
['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O']


2-4) 업소명 + 스타일 + 리뷰문의어미



헤어코코 레이어드컷 거짓없는 후기 알수있나요
['헤어', '코코', '레이어드', '컷', '거짓', '없', '는', '후기', '알', '수', '있', '나요']
['B-ORG', 'I-ORG', 'B-STY', 'I-STY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


2-5) 장소 + 업소명 + 위치문의어미


...


성북구 청담에 오소소헤어 쉐도우펌 거짓없는 평 있는지 궁금해
['성북구', '청담', '에', '오소소', '헤어', '쉐', '도우', '펌', '거짓', '없', '는', '평', '있', '는지', '궁금', '해']
['B-LOC', 'I-LOC', 'O', 'B-ORG', 'I-ORG', 'B-STY', 'I-STY', 'I-STY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


2-9) 지하철역 + 업소명 + 위치문의어미



학동역에서 로이드밤 개봉역점 상세주소 어디일까요
['학동역', '에서', '로이드', '밤', '개봉', '역점', '상세', '주소', '어디', '일까요']
['B-SBW', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O']


2-10) 지하철역 + 업소명 + 가격문의어미



광명사거리역에서 준오헤어 방배역점 가격 알려주세요
['광명사거리역', '에서', '준오', '헤어', '방배', '역점', '가격', '알려', '주', '세요']
['B-SBW', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O']


2-11) 지하철역 + 업소명 + 리뷰문의어미



녹천역에서 차홍룸 용산점 진짜반응 있는지
['녹천역', '에서', '차홍', '룸', '용', '산점', '진짜', '반응', '있', '는지']
['B-SBW', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O']


2-12) 지하철역 + 업소명 + 스타일 + 리뷰문의어미



동대입구역에서 하이살롱 C컬펌 진실된 평가 뭐였어요
['동대입구역', '에서', '하이', '살롱', 'C', '컬', '펌', '진실', '된', '평가', '뭐', '였어요']
['B-SBW', 'O', 'B-ORG', 'I-ORG', 'B-STY', 'I-STY', 'I-STY', 'O', 'O', 'O', 'O', 'O']




In [88]:
len(q_questions), len(q_mecabs), len(q_tags)

(1000000, 1000000, 1000000)

In [89]:
df_r_questions = DataFrame({
      'question' : r_questions
    , 'mecab' : r_mecabs
    , 'BIO_tag' : r_tags
})

df_r_questions

,question,mecab,BIO_tag
0,그레이스펌 잘하는 좋은 미용실 뭐예요,"[그레이스, 펌, 잘, 하, 는, 좋, 은, 미용실, 뭐, 예요]","[B-STY, I-STY, O, O, O, O, O, O, O, O]"
1,글램펌 잘하는 좋은 데 물어볼라는데요,"[글램, 펌, 잘, 하, 는, 좋, 은, 데, 물, 어, 볼, 라는, 데, 요]","[B-STY, I-STY, O, O, O, O, O, O, O, O, O, O, O..."
2,C컬펌 잘하는 지리는 데 어디냐,"[C, 컬, 펌, 잘, 하, 는, 지리, 는, 데, 어디, 냐]","[B-STY, I-STY, I-STY, O, O, O, O, O, O, O, O]"
3,호일펌 잘하는 괜찮은 미용실 어뎁니까,"[호일, 펌, 잘, 하, 는, 괜찮, 은, 미용실, 어, 뎁니까]","[B-STY, I-STY, O, O, O, O, O, O, O, O]"
4,에어펌 잘하는 같이오고 싶은 미용실 여쭤보려하는데요,"[에, 어, 펌, 잘, 하, 는, 같이, 오, 고, 싶, 은, 미용실, 여쭤, 보,...","[B-STY, I-STY, I-STY, O, O, O, O, O, O, O, O, ..."
...,...,...,...
999995,반포역에서 바로 보이는 깔끔한 오지는 데 궁금합니다,"[반, 포역, 에서, 바로, 보이, 는, 깔끔, 한, 오지, 는, 데, 궁금, 합니다]","[B-SBW, I-SBW, O, B-ADJ, I-ADJ, I-ADJ, B-ADJ, ..."
999996,논현역에서 바로 근처에 가성비 좋은 헤어 맛집 찾아줘,"[논현역, 에서, 바로, 근처, 에, 가성, 비, 좋, 은, 헤어, 맛집, 찾, 아...","[B-SBW, O, B-ADJ, I-ADJ, I-ADJ, B-ADJ, I-ADJ, ..."
999997,논현역에서 근처에 매일 소독하는 아는 미용실 알고 계신가요,"[논현역, 에서, 근처, 에, 매일, 소독, 하, 는, 아, 는, 미용실, 알, 고...","[B-SBW, O, B-ADJ, I-ADJ, B-ADJ, I-ADJ, I-ADJ, ..."
999998,총신대입구역에서 금방 걸어가는 정리정돈이 잘된 지리는 데 알려주세요,"[총신, 대입, 구역, 에서, 금방, 걸어가, 는, 정, 리정, 돈, 이, 잘, 된...","[B-SBW, I-SBW, I-SBW, O, B-ADJ, I-ADJ, I-ADJ, ..."


In [98]:
df_q_questions = DataFrame({
      'question' : q_questions
    , 'mecab' : q_mecabs
    , 'BIO_tag' : q_tags
})

df_q_questions

,question,mecab,BIO_tag
0,지노헤어 신당점 어디있나요,"[지노, 헤어, 신, 당점, 어디, 있, 나요]","[B-ORG, I-ORG, I-ORG, I-ORG, O, O, O]"
1,이훈ALL30000 낙성대점 상세주소 어데로가는교,"[이훈, ALL, 30000, 낙성, 대점, 상세, 주소, 어데로, 가, 는교]","[B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O..."
2,my name is 영웅. 헤어&탈모 상세주소 어디예요,"[my, name, is, 영웅, ., 헤어, &, 탈모, 상세, 주소, 어디, 예요]","[B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-ORG, I-O..."
3,더헤어 중랑점 자세한 위치 말해줘,"[더, 헤어, 중, 랑, 점, 자세, 한, 위치, 말, 해, 줘]","[B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O..."
4,에이난 홍대점 자세한 주소 어디인데요,"[에이, 난, 홍대, 점, 자세, 한, 주소, 어디, 인데요]","[B-ORG, I-ORG, I-ORG, I-ORG, O, O, O, O, O]"
...,...,...,...
999995,삼양역에서 김지연살롱에이 스타일바이 내츄럴펌 진짜반응 있는지 궁금해,"[삼, 양역, 에서, 김지연, 살롱, 에, 이, 스타일, 바이, 내츄럴, 펌, 진짜...","[B-SBW, I-SBW, O, B-ORG, I-ORG, I-ORG, I-ORG, ..."
999996,신금호역에서 아가페헤어살롱 레이어드컷 반응 알려주나요,"[신금, 호역, 에서, 아가페, 헤어, 살롱, 레이어드, 컷, 반응, 알려, 주, 나요]","[B-SBW, I-SBW, O, B-ORG, I-ORG, I-ORG, B-STY, ..."
999997,삼각지역에서 플랜에이치 홍대점 해병대머리 솔직후기 여쭤볼려고,"[삼각, 지역, 에서, 플랜, 에, 이치, 홍대, 점, 해병대, 머리, 솔직, 후기...","[B-SBW, I-SBW, O, B-ORG, I-ORG, I-ORG, I-ORG, ..."
999998,청담역에서 히즈모 동부이촌점 히피펌 후기 뭐뭐있어요,"[청담역, 에서, 히즈, 모, 동부, 이촌, 점, 히피, 펌, 후기, 뭐, 뭐, 있...","[B-SBW, O, B-ORG, I-ORG, I-ORG, I-ORG, I-ORG, ..."


In [99]:
df_questions = pd.concat([df_r_questions, df_q_questions], ignore_index = True)
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   question  object
 1   mecab     object
 2   BIO_tag   object
dtypes: object(3)
memory usage: 45.8+ MB


In [100]:
df_questions.drop_duplicates(subset='question', inplace=True)
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1990408 entries, 0 to 1999999
Data columns (total 3 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   question  object
 1   mecab     object
 2   BIO_tag   object
dtypes: object(3)
memory usage: 60.7+ MB


In [101]:
from sklearn.utils import shuffle

df_questions = shuffle(df_questions)
df_questions.index = range(len(df_questions))

df_questions.head()

,question,mecab,BIO_tag
0,금천 가락1동에 이철헤어커커 신림포도몰점 히메컷 진짜리뷰 뭔고,"[금천, 가락, 1, 동, 에, 이철, 헤어, 커, 커, 신림, 포도, 몰, 점, ...","[B-LOC, I-LOC, I-LOC, I-LOC, O, B-ORG, I-ORG, ..."
1,자연스러워 보이는 볼륨감이 살아있는 실패하지않는 데 어데인지아나,"[자연, 스러워, 보이, 는, 볼륨, 감, 이, 살, 아, 있, 는, 실패, 하, ...","[B-ADJ, I-ADJ, I-ADJ, I-ADJ, B-ADJ, I-ADJ, I-A..."
2,용산 이문동에 쾌적한 하고싶은 미용실 알려줄 수 있나요,"[용산, 이문동, 에, 쾌적, 한, 하, 고, 싶, 은, 미용실, 알려, 줄, 수,...","[B-LOC, I-LOC, O, B-ADJ, I-ADJ, O, O, O, O, O,..."
3,상월곡역에서 고운헤어 가본 사람들 리뷰 알아보는데요,"[상월곡역, 에서, 고운, 헤어, 가본, 사람, 들, 리뷰, 알아보, 는데요]","[B-SBW, O, B-ORG, I-ORG, O, O, O, O, O, O]"
4,내부가 예쁜 투블럭펌 잘하는 잠실역에서잘하는 헤어샵 있다면서요,"[내부, 가, 예쁜, 투, 블럭, 펌, 잘, 하, 는, 잠실역, 잘, 하, 는, 헤...","[B-ADJ, I-ADJ, I-ADJ, B-STY, I-STY, I-STY, O, ..."


In [102]:
df_questions.to_csv('./output_data/QuestionsMorphsBIOTags.csv', sep=',', encoding='utf-8-sig')